In [ ]:
!pip install -U --no-cache-dir "torch>=2.3" "huggingface_hub>=0.23.0"
!pip uninstall -y diffusers
!pip install --no-cache-dir "git+https://github.com/huggingface/diffusers.git@main"
!pip install --upgrade --no-cache-dir "transformers>=4.44" "accelerate>=0.34" safetensors peft bitsandbytes sentencepiece safetensors protobuf

In [ ]:
import torch
from diffusers import DiffusionPipeline, FluxTransformer2DModel, AutoencoderKL
from transformers import CLIPTextModel, T5EncoderModel, AutoTokenizer
dtype = torch.bfloat16


ckpt =  "/workspace/models/flux-dev"
#ckpt =  "/workspace/models/flux-kontext-dev"

clip = CLIPTextModel.from_pretrained(ckpt, subfolder="text_encoder", torch_dtype=dtype)
t5   = T5EncoderModel.from_pretrained(ckpt, subfolder="text_encoder_2", torch_dtype=dtype)
trf  = FluxTransformer2DModel.from_pretrained(ckpt, subfolder="transformer", torch_dtype=dtype)
vae  = AutoencoderKL.from_pretrained(ckpt, subfolder="vae", torch_dtype=dtype)

# Токенайзеры
tok_clip = AutoTokenizer.from_pretrained(ckpt, subfolder="tokenizer", use_fast=False)
tok_t5   = AutoTokenizer.from_pretrained(ckpt, subfolder="tokenizer_2", use_fast=True)

# 2) Собираем пайплайн, подменяя компоненты
pipe = DiffusionPipeline.from_pretrained(
    ckpt,
    torch_dtype=dtype,
    transformer=trf,
    vae=vae,
    text_encoder=clip,
    text_encoder_2=t5,
    tokenizer=tok_clip,
    tokenizer_2=tok_t5,
)
pipe.to("cuda")

In [ ]:
# Загружаем только стиль
pipe.load_lora_weights("/workspace/weights/style/checkpoint-1000")

In [ ]:
# Загружаем стиль и объект
pipe.load_lora_weights("/workspace/weights/style/checkpoint-1000", adapter_name="style")
pipe.load_lora_weights("/workspace/weights/Kremlin/checkpoint-800", adapter_name="kreml")

In [ ]:
# Подбираем баланс
pipe.set_adapters(["style", "kreml"], adapter_weights=[0.9, 0.3])

In [ ]:
# В случае image-to-image лучше ставить меньше 1
pipe.set_adapters(["default_0"], adapter_weights=[0.5])

In [ ]:
prompt = "Modern district with tall residential towers, wide sidewalks, cars, two people sleek high-rises, evening lights, cinematic, in <sksartist> style"
for i in range(90, 100):
    image = pipe(prompt, num_inference_steps=36, guidance_scale=3.5, width=1024, height=1024).images[0]
    image.save(f"images/moscow_last_{i}.png")
    print(f"images/moscow_last_{i}.png")

In [ ]:
from diffusers.utils import load_image
pipe.set_adapters(["default_0"], adapter_weights=[1])
img = load_image("./dataset/MSU/2.jpg").convert("RGB")
out = pipe(
    image=img,                              # здесь image уже допустим
    prompt="in <sksartist> style, dramatic bright lighting",
    guidance_scale=2.5,
    num_inference_steps=36,
    width=1024, height=1024,
    generator=torch.Generator().manual_seed(234),
).images[0]
out.save("images/msu_from_orig2_new_dramatic1.png")